In [1]:
import requests
from bs4 import BeautifulSoup
from langchain_openai import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
import os

In [ ]:
os.environ["OPENAI_API_KEY"]= ""
LANGCHAIN_TRACING_V2="true"
LANGCHAIN_API_KEY= ""

In [3]:
def extract_page(url):
    text=requests.get(url)
    soup=BeautifulSoup(text.text,"html.parser")
    return soup.text

In [4]:
def language_detector(query):
    lang_url="https://ddragon.leagueoflegends.com/cdn/languages.json"
    text_langs=extract_page(lang_url).split('"')

    list_langs=[]
    for lang in range(1,len(text_langs),2):
        list_langs.append(text_langs[lang])

    determinate_lang=f"""You are an expert to determinate what language a text is in.
    You are an expert because you always check the whole text to determinate what overall language is used.
    You always provide a single language among the following list: {list_langs}
    You always provide only the language, nothing more.
    Here is a text:
    {query}"""

    lang=llm.invoke(determinate_lang).content
    return lang

In [5]:
def champion_detector(query):
    champ_url="https://ddragon.leagueoflegends.com/cdn/15.23.1/data/en_US/champion.json"
    champ_page=requests.get(champ_url).json()
    champs=[champ for champ in champ_page["data"]]

    determinate_champ=f"""You are an expert to determinate what League Of Legends champions are related to a text.
    You are an expert because you always check the list of League Of Legends champions and then you compare it to the text.
    You provide as many champions as mentionned in the text and present in this list: {champs}
    You always provide the list of related champions as followed: [\"champion_name1\",\"champion_name2\"] with as many names as related.
    Here is a text:
    {query}"""

    champions=llm.invoke(determinate_champ).content.split('"')
    champions_list=[]

    for champ in range(1,len(champions),2):
        champions_list.append(champions[champ])

    return champions_list

In [ ]:
ATTRIBUTES_CHAMP=[
    "allytips",
    "enemytips",
    "tags",
    "partype",
    "info",
    "stats",
    "spells",
    "passive",
    "recommended"
]

def champ_infos(lang,champion):
    champ_url=f"https://ddragon.leagueoflegends.com/cdn/15.23.1/data/{lang}/champion/{champion}.json"
    champ_page=requests.get(champ_url).json()["data"][champion]
    return [champ_page[infos] for infos in champ_page if infos in ATTRIBUTES_CHAMP and champ_page[infos]!=[]]


In [97]:
def champion_agent(query,lang):
    champions=champion_detector(query)
    champions_info=[champ_infos(lang=lang,champion=champ) for champ in champions]
    prompt_champion=f"""You are a League of Legends champion strategy expert.\
    Your role is to provide detailed guidance for specific matchups, including:\
    -how to trade\
    -power spikes\
    -wave management\
    -early / mid / late game plans\
    -counterplay and win conditions\
    Use domain-specific vocabulary and be actionable.\
    Reference knowledge from retrieved documents if available.\
    If data is missing, clearly say so instead of making things up.\
    Here are data about the champions: {champions_info}\
    You must follow the format:\
    How to trade : strategy,\
    Power spikes : power spikes,\
    Wave management : wave management,\
    Early game : early game plan,\
    Mid game : mid game plan,\
    Late game : late game plan,\
    Counterplay and win conditions : counterplay and win conditions,\
    Mistakes to avoid : mistakes to avoid,\
    Here is a query : {query}
    """
    champion_answer=llm.invoke(prompt_champion).content
    return champion_answer

def macro_agent(query):
    macro_prompt=f"""You are a high-level macro strategy coach (Master+ level analytical style).\
    You give strategic plans independent of champion mechanics, focusing on:\
    -rotations, tempo, and resets\
    -objective tradeoffs (Herald, Dragon, Baron)\
    -map control, vision\
    -playing from ahead vs behind\
    Avoid generic advice—be tactical and specific.\
    Here is a query : {query}
    """
    macro_answer=llm.invoke(macro_prompt).content
    return macro_answer

def items_agent(query):
    items_prompt=f"""You are a League of Legends itemization specialist.\
    You provide:\
    -optimal build paths\
    -situational items based on threats (AP, burst, anti-heal, armor)\
    -rune pages\
    -alternate builds depending on gold state (ahead/even/behind)\
    Justify choices using mechanics (not just listing items).\
    Here is a query : {query}
    """
    items_answer=llm.invoke(items_prompt).content
    return items_answer

In [ ]:
def rag_answer(query):
    prompt_split=f"""You are an expert agent in League Of Legends.\
    Your role is to analyse the user query, and transform it into subqueries.\
    When the user query doesn't require subqueries, you only return the user query.\
    You are an expert because you always take all informations contained in the user query to form the subqueries.\
    You never skip informations contained in the user query.\
    You always provide only the subqueries and nothing more.\
    Here is a query: {query}
    """

    subqueries=llm.invoke(prompt_split).content.split("\n")
    subqueries=[sub for sub in subqueries if sub!="Subqueries:"]
    print("\n".join(subqueries))

    for subquery in subqueries:
        print(subquery)
        prompt_router=f"""You are the coordinator agent of a multi-agent League Of Legends system.\
        Your role is to analyse the user query, determine which specialized agents should be consulted, and produce task-specific sub-queries.\
        You do not generate strategic answers yourself.\
        You dispatch requests to the following agents:\
        - "Champion - matchups, laning, combos, power spikes"\
        - "Macro - rotations, objectives, tempo, comeback, map strategy"\
        - "Items - builds, runes, situational items"\
        - "Stats - winrates, pick/ban, meta data"\
        - "Patch - recent balance changes + impact"\
        Your output must follow this format:\
        UsedAgent1 : Subquery,\
        UsedAgent2 : Subquery,\
        ...
        FinalGoal : finalgoal\
        Here is a query : {subquery}
        """
        plan=llm.invoke(prompt_router).content
        champion_query=plan.split("Champion : ")[1].split(",")[0] if len(plan.split("Champion : "))!=1 else None
        macro_query=plan.split("Macro : ")[1].split(",")[0] if len(plan.split("Macro : "))!=1 else None
        items_query=plan.split("Items : ")[1].split(",")[0] if len(plan.split("Items : "))!=1 else None
        stats_query=plan.split("Stats : ")[1].split(",")[0] if len(plan.split("Stats : "))!=1 else None
        patch_query=plan.split("Patch : ")[1].split(",")[0] if len(plan.split("Patch : "))!=1 else None
        finalGoal=plan.split("FinalGoal : ")[1]

        lang=language_detector(query)

        #champion_answer=champion_agent(champion_query,lang) if champion_query is not None else None
        
        #macro_answer=macro_agent(macro_query) if macro_query is not None else None

        items_answer=items_agent(items_query) if items_query is not None else None
    
        stats_answer=stats_agent(stats_query) if stats_query is not None else None

        patch_answer=patch_agent(patch_query) if patch_query is not None else None
        print(plan)
    return None

emb=OpenAIEmbeddings()
llm=ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.0,
    max_tokens=500
)
print(rag_answer("Je joue Orianna mid et je suis en train de perdre contre Akali, on est à 15 min de jeu que faire? Il y a également un zed jungle qui me dérange en face."))

1. Comment contrer Akali en tant que Orianna mid?
2. Comment gérer la présence de Zed jungle en tant que Orianna mid?
1. Comment contrer Akali en tant que Orianna mid?
As a high-level macro strategy coach, I would recommend focusing on the following key points:

1. Rotations, Tempo, and Resets:
- Encourage your team to constantly be aware of the map and make proactive rotations to gain advantages in objectives or map control.
- Emphasize the importance of maintaining tempo by consistently pressuring the enemy team and controlling the pace of the game.
- Teach your team the value of resetting efficiently after a teamfight or objective take to ensure they are in the best position for the next play.

2. Objective Tradeoffs:
- Prioritize objectives based on the current state of the game and the strengths of your team composition.
- Help your team understand when it is worth trading objectives and when it is better to contest or prioritize a specific objective.
- Encourage your team to comm